In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import azureml.core
azureml.core.__version__

# to upgrade:
# pip install --upgrade azureml-sdk

'1.0.33'

In [27]:
from azureml.core import Workspace, ComputeTarget
from azureml.core.compute import AmlCompute
from azureml.core.model import Model

# Create a new AML instance

Run this notebook to create a new instance of Azure Machine Learning to support a new instance of the batch processing API.

Azure Machine Learning SDK for Python documentation: https://docs.microsoft.com/en-us/python/api/overview/azure/ml/intro?view=azure-ml-py

Modify params in this section:

In [30]:
# Workspace name has to be between 2 and 32 characters of letters and numbers.
workspace_name = 'camera_trap_aml_workspace_2'

compute_name = 'camera-trap-com'  # 2 to 16 chars
max_nodes = 16
min_nodes = 0  # set to 0 to allow the cluster to completely deallocate
idle_seconds_before_scaledown = 120

# set these credentials so you can ssh into the nodes to debug if needed. There's no way to set this after this step!
admin_username = 'admin'
admin_user_password = 'default_password'

assert len(workspace_name) < 33
assert len(compute_name) < 17

# models you'd like to register. The .pb files need to be local
models = [
    {
        'name': 'megadetector_v3',
        'description': 'megadetector version 3',
        'path': 'path/megadetector_v3.pb'
    },
    {
        'name': 'megadetector_v2',
        'description': 'megadetector version 2',
        'path': 'path/frozen_inference_graph.pb'
    }
]

Modify the following section if you'd like to create the AML workspace in your own subscription.

In [ ]:
# keep these the same for internal deployments
subscription_id = ''  # fill out the subscription_id
resource_group = 'camera_trap_api_rg'
location = 'eastus'

# used by the workspace to save run outputs, code, logs etc
storage_account='subscriptions/{}/resourcegroups/{}/providers/microsoft.storage/storageaccounts/cameratrstorageiuquhxss'.format(
    subscription_id, resource_group)  
key_vault = 'subscriptions/{}/resourcegroups/{}/providers/microsoft.keyvault/vaults/cameratrkeyvaulthblvewsj'.format(
    subscription_id, resource_group)
app_insights = 'subscriptions/{}/resourcegroups/{}/providers/microsoft.insights/components/cameratrinsightsiqgcufll'.format(
    subscription_id, resource_group)
container_registry = 'subscriptions/{}/resourcegroups/{}/providers/microsoft.containerregistry/registries/cameratracrsppftkje'.format(
    subscription_id, resource_group)

## Log in to Azure on the CLI

After logging in, make sure the default account shown here is the subscription specified above.

```
az account show
```

Otherwise do 

```
az account set <subscription-id>
```

## Create AML workspace

[Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace%28class%29?view=azure-ml-py#create-name--auth-none--subscription-id-none--resource-group-none--location-none--create-resource-group-true--friendly-name-none--storage-account-none--key-vault-none--app-insights-none--container-registry-none--default-cpu-compute-target-none--default-gpu-compute-target-none--exist-ok-false--show-output-true-)

In [9]:
# takes ~10 seconds and it will show a message "Deployed Workspace with name..." if successful.

workspace = Workspace.create(workspace_name, 
                             auth=None, # If None the default Azure CLI credentials will be used or the API will prompt for credentials
                             subscription_id=subscription_id, 
                             resource_group=resource_group, 
                             location=location, 
                             create_resource_group=False, 
                             friendly_name=None, 
                             storage_account=storage_account, 
                             key_vault=key_vault, 
                             app_insights=app_insights, 
                             container_registry=container_registry,
                             exist_ok=False, 
                             show_output=True)

Deploying Workspace with name camera_trap_aml_workspace_2.
Deployed Workspace with name camera_trap_aml_workspace_2.


In [12]:
# optinally save the workspace's config to a text file - not necessary - you can identify a workspace without this file
# workspace.write_config(path='')

## Create a compute target in the workspace
if it doesn't yet exist

[Documentation](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.computetarget?view=azure-ml-py#create-workspace--name--provisioning-configuration-)

In [24]:
if compute_name in workspace.compute_targets:
    compute_target = workspace.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('Found existing compute target with this name. You can just use it.' + compute_name)
else:
    print('No existing compute target found, creating...')
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_NC6S_V3',
        min_nodes=min_nodes, max_nodes=max_nodes,
        idle_seconds_before_scaledown=idle_seconds_before_scaledown,
        admin_username=admin_username, admin_user_password=admin_user_password)
    
    compute_target = AmlCompute.create(workspace, name=compute_name, provisioning_configuration=compute_config)
    compute_target.wait_for_completion(show_output = True)

No existing compute target found, creating...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


## Register and upload model files

You could ask each job to load the model from blob storage, but registering them with the AML workspace allows you to switch models on the go better.

This takes a while depending on the size of the files.

In [31]:
for m in models:
    model = Model.register(workspace, m['path'], m['name'], description=m['description'])

Registering model megadetector_v3
Registering model megadetector_v2


## Authorize our application to access this AML workspace

We created the workspace above by authenticating to our subscription on the CLI. When our API needs to access the AML instance, it has to authenticate as an application. We now need the AML workspace to give that application access to the it.

Instructions for doing this is [here](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb). 

- If you already have an application that the API instance will be using, go to step starting with "Finally, you need to give the service principal permissions to access your workspace".